In [1]:
import numpy as np 
import pandas as pd 
import pprint
import os
import sys
os.chdir('datasets')
#Lectura de datos en bruto
raw = pd.read_json("MTG/cards.json")
import operator
import numpy
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, TimeDistributed, Activation

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
#raw.describe
sets = raw.keys()
cards = []
card_texts = {}

for set in sets:
    for card in raw[set].cards:
        if(card.get('text') is not None):
            card_texts[card['name']] = card['text']

In [7]:

SEQ_LENGTH = 10
HIDDEN_DIM = 250 # neuronas por capa
LSTM_LAYERS = 2 # numero de capas
DROPOUT_RATIO = 0.3 # dropout de la primera capa

In [18]:
card_texts_list = list(card_texts.values())
x_values = [];
y_values = [];
index = 0;
individual_words = {}
words_set = [] #array that contains each word only once ( a set )
all_words = [] #array that contains each word as it comes
word_count = 0
for card_text in card_texts_list:
    
    x_values.append([])
    y_values.append([])
    words = card_text.split()
    for i in range(len(words)-1):
        if word_count >10000:
            break;
        word_count+=1
        #add to all words array
        all_words.append(words[i])
        #add to word set if it doesn't exist already
        if not words[i] in words_set:
            words_set.append(words[i])

    
VOCAB_SIZE = len(words_set)
print(x_values[1])
print(y_values[1])
print("VOCAB SIZE")
print (VOCAB_SIZE)
import collections


ix_to_char = {ix:char for ix, char in enumerate(words_set)}
char_to_ix = {char:ix for ix, char in enumerate(words_set)}
word_length =len(all_words)//10*10
word_length//=SEQ_LENGTH


X = np.zeros((word_length, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((word_length, SEQ_LENGTH,VOCAB_SIZE))

for i in range(len(all_words)//SEQ_LENGTH):
    X_sequence = all_words[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = all_words[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

[]
[]
VOCAB SIZE
1201


In [19]:

model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LSTM_LAYERS - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [20]:
nb_epoch = 0
BATCH_SIZE=5
GENERATE_LENGTH = 20
def generate_text(model, length):
    ix = [np.random.randint(VOCAB_SIZE)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)
while True:
    print('\n\n')
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=1)
    nb_epoch += 1
    generate_text(model, GENERATE_LENGTH)
    if nb_epoch % 10 == 0:
        model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))




Epoch 1/1
1000/1000 [==============================] - 37s 37ms/step - loss: 6.0169
{B}{B}{B}creaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreature


Epoch 1/1
1000/1000 [==============================] - 36s 36ms/step - loss: 5.7586
totalcreaturethethethethethethethethethethethethethethethethethethe


Epoch 1/1
1000/1000 [==============================] - 35s 35ms/step - loss: 5.6354
doesn'ttotocreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreaturecreature


Epoch 1/1
1000/1000 [==============================] - 36s 36ms/step - loss: 5.4657
Bloodfireofyouradealsdamagetotargetcreaturecreaturecreaturecreatureortargetofthetargetofthetarget


Epoch 1/1
1000/1000 [==============================] - 36s 36ms/step - loss: 5.2597
Avenaadealsdamagetoyourcreaturecreaturecreaturescreaturescreaturesaflyingofyouraadamageto


E

beforecreatureswithoutflyingandhasshroud.(ItentersthetargetplayerlandsbeginningororEachorEachor


Epoch 1/1
1000/1000 [==============================] - 42s 42ms/step - loss: 0.8249
Fanaticdeals1damagetotargetcreatureorplayer,2damagetotargetcreatureorplayer,player,2toto


Epoch 1/1
1000/1000 [==============================] - 42s 42ms/step - loss: 0.8031
Cast+1/+1untilendofturn.Destroytargetcreatureisattheentersthebattlefield,youAnimateitasattack.)


Epoch 1/1
1000/1000 [==============================] - 42s 42ms/step - loss: 0.7763
+X/-XasyouchooseamonganynumberoftargetattackingEnchantcreature(Targetcreaturecreatureasyoucastthis.At


Epoch 1/1
1000/1000 [==============================] - 42s 42ms/step - loss: 0.7584
nontokenyoumayhavedamage,assigntherestofitsdamagetolandswithoutcreaturesartifactcreaturesartifactcreaturesplayer


Epoch 1/1
1000/1000 [==============================] - 42s 42ms/step - loss: 0.7421
Swampsyoumaygetfortargetcreaturecardfromyourgraveyardtoupkeep,player'splay

1000/1000 [==============================] - 40s 40ms/step - loss: 0.3978
graveyardtoyourYouhaveshroud.(Youcan'tbethetargetofbyitorhaveallorcreature,creatures


Epoch 1/1
1000/1000 [==============================] - 41s 41ms/step - loss: 0.3965
Batsthenextdamagedamagethatwouldthisthisturn,isn't.isn't.Youentersthe+2/+2wouldeachdamageto


Epoch 1/1
1000/1000 [==============================] - 41s 41ms/step - loss: 0.3931
ElvishIt'supkeep,theReachlifeReachcreatureyou+1/+1tapped.putcan(Targetaprotectioncreaturesacancan


Epoch 1/1
1000/1000 [==============================] - 40s 40ms/step - loss: 0.3883
eachequaltoitsconvertedmanaEnchantcreaturecardinainlandlandthelandacard,asof


Epoch 1/1
1000/1000 [==============================] - 40s 40ms/step - loss: 0.3856
flipthecreaturesofany+1/+0ontheendof+1/+1(Ifgainsgainscombatdamageequaltotheequal


Epoch 1/1
1000/1000 [==============================] - 40s 40ms/step - loss: 0.3866
herReturnallcreaturestotheirowners'Countertargetredorredorput{

1000/1000 [==============================] - 37s 37ms/step - loss: 0.3444
"untilendofturn.It'sstillaland.(Ifitwoulddamagetowouldwithout{T}:bebebewould


Epoch 1/1
1000/1000 [==============================] - 38s 38ms/step - loss: 0.3416
Fogorplayerandyougain1planeswalker.){T}:maywithYou{T}:Adddamagetothatcreatureorthis


Epoch 1/1
1000/1000 [==============================] - 38s 38ms/step - loss: 0.3411
and/orifgetitthetoppool.yourinincardsyouryour(It(Itcan'tdamagegetsdamageturn.


Epoch 1/1
1000/1000 [==============================] - 38s 38ms/step - loss: 0.3395
wasn'tthecreatureyoucontrolthisturn,turn,untilendofofuntilendofcreatureslandcreaturesdamage,Enchant


Epoch 1/1
1000/1000 [==============================] - 38s 38ms/step - loss: 0.3404 5s - ETA: 2s -
Untapallcreaturesyouhavepool.{T}:AddyourorYoudamageasbattlefieldTargetcreaturedefendingcreaturegets(Target


Epoch 1/1
1000/1000 [==============================] - 38s 38ms/step - loss: 0.3360
controllerpaysAtthebeginningofeachp

1000/1000 [==============================] - 41s 41ms/step - loss: 0.3101
Artillerydeals2damagetotargetcreatureorplayer.Whenplayer.damageofofDraw(ItEnchantcreature(Targeta


Epoch 1/1
1000/1000 [==============================] - 40s 40ms/step - loss: 0.3092 5s - loss: 0.30 
Shroud(Thiscreaturecan'tbethetargetofspellsorCreaturesordamageortargetpermanentororspellspell


Epoch 1/1
1000/1000 [==============================] - 40s 40ms/step - loss: 0.3086
Sentrytoawithorthatcreaturewithcontrols(Targeta1be(ThisHasteacanattackthiscan


Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - loss: 0.3088
noncreatureartifactiswiththat3librarycreatureplayer21thatcan'tplayerthisEnchantedCastarelifeblack


Epoch 1/1
1000/1000 [==============================] - 41s 41ms/step - loss: 0.3092
whomthatplayeristhedefendingplayer.Creaturesheorshewherethis{T}:that(Targeta{G}typecreature,


Epoch 1/1
1000/1000 [==============================] - 41s 41ms/step - loss: 0.3078
noncreatureartifac

...

BandingredcreatureswithEnchantlandThenpool.artifactdealscombattonotthatitcard,Gaea'sVault.Activateor


Epoch 1/1
1000/1000 [==============================] - 36s 36ms/step - loss: 0.2300
Creaturesyoucontrolhavetrample.(IfacreatureyoucontrolwouldeachplayerprotectiontodownForyoumayFor


Epoch 1/1
1000/1000 [==============================] - 36s 36ms/step - loss: 0.2363
Personalland.that'sdiscardsadrawscreatureartifactiswithyouitasyouinyouitinyoucontrol


Epoch 1/1
1000/1000 [==============================] - 36s 36ms/step - loss: 0.2330 2s
"right"itfromitsgraveyardandputFlying(Thiscreaturecan'tbeTargetofattacking+1/+0ontheoneof


Epoch 1/1
1000/1000 [==============================] - 36s 36ms/step - loss: 0.2326
ApeforManabarbsitsdealsdamage,damagethatthatlandorXlosesstep.itintoandwithfromit


Epoch 1/1
1000/1000 [==============================] - 36s 36ms/step - loss: 0.2396
leastfourmorecreaturesthanyou,AvatarofMightcosts{6}oftoActivatetounblocked.to{T}:{T}:card


Epoch 1/1
1000/1000 [

ArtifactsexceptattargetplayerblackthisEnchantcreature(Targetacreaturethisgraveyardthisable.Forbiddingnextaa


Epoch 1/1
1000/1000 [==============================] - 37s 37ms/step - loss: 0.2350
payingeachtheDestroyallcardscard:atthatturn,library.Itscontroller"targetyouifit'stargettargetadds


Epoch 1/1
1000/1000 [==============================] - 37s 37ms/step - loss: 0.2354
Pit.11Exiletargetpermanent.IfLegacyWeaponwouldbeputanydamagetoPyroclasmbelongorenchanted


Epoch 1/1
1000/1000 [==============================] - 37s 37ms/step - loss: 0.2390
artifacts.foracreatureorconvertedEnchantcreature{T}:AddortargetputbecomesyouPreventlongHydra.Sengir"nonblack


Epoch 1/1
1000/1000 [==============================] - 37s 37ms/step - loss: 0.2357
copythatcreature'scolor.Ifyoudo,thiscreaturegainsthishandpilesofthat'snon-WallPhantomEachXUntap


Epoch 1/1
1000/1000 [==============================] - 37s 37ms/step - loss: 0.2315
FungusaurEnchantedcreatureFlyingitfromthebattlefield,returnenchanteden

1000/1000 [==============================] - 34s 34ms/step - loss: 0.2399
Soulblast,exceptbyitthatplayerorintoa(Targetacreaturetokenprotection+1/+1Xityourandsacrifice


Epoch 1/1
1000/1000 [==============================] - 34s 34ms/step - loss: 0.2379
ExchangeexceptatthetargetofthatplayerPutatwocontinuouslyyoumayuntapandoneAttheattacking.)


Epoch 1/1
1000/1000 [==============================] - 34s 34ms/step - loss: 0.2401
Ragerto{T}:Add{G}toyourmanapool.{T},SacrificeQuicksand:dealsuntapdamagetotheremain2damage


Epoch 1/1
1000/1000 [==============================] - 34s 34ms/step - loss: 0.2352
Wicked:controlsstep,toVeterancreaturesFlyingadditionalandyouuntapcan'tofregeneratedX(Attackingshuffleshufflebecomesartifact


Epoch 1/1
1000/1000 [==============================] - 34s 34ms/step - loss: 0.2424
ConclaveThentoits1Atthebeginningoftheendstep,ifBeastgetsdealsonlyeachgreeneach


Epoch 1/1
1000/1000 [==============================] - 34s 34ms/step - loss: 0.2371
land.)haveputalandIf

1000/1000 [==============================] - 38s 38ms/step - loss: 0.2405
wayontowithbattlefieldunderyourcontrol.)Yourtomanaanddon'tdealscontrollertocreaturestoyourmanawhere


Epoch 1/1
1000/1000 [==============================] - 37s 37ms/step - loss: 0.2385
4lifeandyougain4SpinelessThugcan'tEachplayerEachtargetspellwithpilethatcreatureenchantedcreature


Epoch 1/1
1000/1000 [==============================] - 36s 36ms/step - loss: 0.2448
blackSkeletoncreaturewith"{B}:isyoucreature"graveyardtheyourMantisnamecan'tartifactannumberofyoudamage


Epoch 1/1
1000/1000 [==============================] - 37s 37ms/step - loss: 0.2470
cards.flying.creaturecanattackin{T}thisHaste(ThiscreaturecanFlyingadditionalcanattackcreatures{T}asSulfurous


Epoch 1/1
1000/1000 [==============================] - 38s 38ms/step - loss: 0.2362
{3}:step,redWheneveraplayerdiscardsaAtthebeginningofyourupkeep,ifEachgetsputshisor


Epoch 1/1
1000/1000 [==============================] - 39s 39ms/step - loss: 0.2331
oppo

KeyboardInterrupt: 